1. Establish a connection between Python and the Sakila database.

In [6]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine, text
import getpass  # To get the password without showing the input
password = getpass.getpass()

 ········


In [3]:
bd = "sakila"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+bd
engine = create_engine(connection_string)

2. Write a Python function called `rentals_month` that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

	- `engine`: an object representing the database connection engine to be used to establish a connection to the Sakila database.
	- `month`: an integer representing the month for which rental data is to be retrieved.
	- `year`: an integer representing the year for which rental data is to be retrieved.

	The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [7]:
def rentals_month(engine, year, month):
    with engine.connect() as connection:
        txt = f'''SELECT *
                  FROM rental
                  WHERE year(rental_date) = {year} AND month(rental_date) = {month};;'''
        query = text(txt)
        result = connection.execute(query)
        return pd.DataFrame(result.all())

In [85]:
df = rentals_month(engine, 2005, 6)
df

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
2306,3465,2005-06-21 22:10:01,1488,510,2005-06-30 21:35:01,1,2006-02-15 21:30:53
2307,3466,2005-06-21 22:13:33,371,226,2005-06-25 21:01:33,2,2006-02-15 21:30:53
2308,3467,2005-06-21 22:19:25,729,543,2005-06-27 00:03:25,2,2006-02-15 21:30:53
2309,3468,2005-06-21 22:43:45,2899,100,2005-06-30 01:49:45,1,2006-02-15 21:30:53


3. Develop a Python function called `rental_count_month` that takes the DataFrame provided by `rentals_month` as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year. 

	The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".


	*Hint: Consider making use of pandas [groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)*

In [109]:
def rental_count_month(engine, year, month):
    df = rentals_month(engine, year, month)
    counts = list(df.groupby('customer_id')['rental_id'].count())
    customer_ids = list(df.groupby('customer_id')['rental_id'].count().index)
    new_df = pd.DataFrame({
        "customer_id":customer_ids,
        f"rentals_{month}_{year}":counts
    })
    return new_df

In [110]:
# easier
def rental_count_month(engine, year, month):
    df = rentals_month(engine, year, month)
    df_count = df.groupby('customer_id', as_index=False)['rental_id'].count().rename(columns={"rental_id":f"rentals_{month}_{year}"})
    return df_count

In [111]:
rental_count_month(engine, 2005, 6)

,customer_id,rentals_6_2005
0,1,7
1,2,1
2,3,4
3,4,6
4,5,5
...,...,...
585,595,2
586,596,2
587,597,3
588,598,1


4. Create a Python function called `compare_rentals` that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. 
The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [112]:
def compare_rentals(df1, df2):
    new_df = df1.merge(df2, on="customer_id")
    new_df["difference"] = new_df[new_df.columns[2]] - new_df[new_df.columns[1]]
    return new_df

In [113]:
df1 = rental_count_month(engine, 2005, 5)
df2 = rental_count_month(engine, 2005, 6)
compare_rentals(df1, df2)

,customer_id,rentals_5_2005,rentals_6_2005,difference
0,1,2,7,5
1,2,1,1,0
2,3,2,4,2
3,5,3,5,2
4,6,3,4,1
...,...,...,...,...
507,594,4,6,2
508,595,1,2,1
509,596,6,2,-4
510,597,2,3,1
